In [1]:
import keras
import random, os
from keras.layers import Activation, Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten
from keras.models import Model, Sequential
from keras import backend as K
from glob import glob
import numpy as np
import re
from keras.utils.vis_utils import model_to_dot
import random
import matplotlib.pyplot as plt
from scipy.spatial import distance
from IPython.display import Image
import PIL

Using TensorFlow backend.


In [2]:
batch_size = 256
num_classes = 6
epochs = 1000
img_rows, img_cols = 28, 28
label = ["circle", "square", "triangle", "horizontal", "vertical", "diagonal"]

In [3]:
def load_data():
    tr_path = './dataset/trains/'
    te_path = './dataset/tests/'
    va_path = './dataset/vals/'
    
    (tr_sets, tr_labels, _) = get_datasets_labels(tr_path)
    (te_sets, te_labels, test_labels_with_num) = get_datasets_labels(te_path)
    (va_sets, va_labels, _) = get_datasets_labels(va_path)
    
    
    tr_sets = np.array(tr_sets)
    te_sets = np.array(te_sets)
    va_sets = np.array(va_sets)
    tr_labels = np.array(tr_labels)
    te_labels = np.array(te_labels)
    va_labels = np.array(va_labels)
    test_labels_with_num = np.array(test_labels_with_num)
    return tr_sets, tr_labels, te_sets, te_labels, va_sets, va_labels, test_labels_with_num

In [4]:
def get_datasets_labels(path):
	files = os.listdir(path)
	labels = []
	lbs = []
	sets = []
	for file in files:
		pic = PIL.Image.open(path + file)
		lb = file.replace(re.findall("\d+", file)[0], '').replace('.png', '') 
		forlb = file.replace('.png', '') 
		
		labels.append(label.index(lb))
		lbs.append(forlb)
		
		pic = np.array(pic)
		sets.append(pic)
	return sets, labels, lbs

In [5]:
(x_train, y_train, x_test, y_test, x_val, y_val, test_labels_with_num) = load_data()
y_test1 = y_test
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255.
x_test /= 255.
x_val /= 255.

In [68]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))
#

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(64, activation = 'relu'))

#model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

In [69]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 28, 28, 128)       1280      
_________________________________________________________________
activation_42 (Activation)   (None, 28, 28, 128)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_43 (Activation)   (None, 14, 14, 128)       0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 7, 7, 64)          73792     
__________

In [70]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [71]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, validation_data=(x_val, y_val))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1800 samples, validate on 1200 samples
Epoch 1/1000
1800/1800 [==============================] - 1s 323us/step - loss: 1.7929 - acc: 0.1728 - val_loss: 1.7919 - val_acc: 0.1725
Epoch 2/1000
1800/1800 [==============================] - 0s 61us/step - loss: 1.7921 - acc: 0.1722 - val_loss: 1.7927 - val_acc: 0.1667
Epoch 3/1000
1800/1800 [==============================] - 0s 61us/step - loss: 1.7927 - acc: 0.1556 - val_loss: 1.7929 - val_acc: 0.1667
Epoch 4/1000
1800/1800 [==============================] - 0s 59us/step - loss: 1.7919 - acc: 0.1817 - val_loss: 1.7945 - val_acc: 0.1667
Epoch 5/1000
1800/1800 [==============================] - 0s 57us/step - loss: 1.7944 - acc: 0.1550 - val_loss: 1.7917 - val_acc: 0.1667
Epoch 6/1000
1800/1800 [==============================] - 0s 56us/step - loss: 1.7920 - acc: 0.1683 - val_loss: 1.7921 - val_acc: 0.1667
Epoch 7/1000
1800/1800 [==============================] - 0s 60us/step - loss: 1.7921 - acc: 0.1656 - val_loss: 1.7918 - val_acc:

1800/1800 [==============================] - 0s 55us/step - loss: 0.5297 - acc: 0.7839 - val_loss: 0.9358 - val_acc: 0.6558
Epoch 61/1000
1800/1800 [==============================] - 0s 53us/step - loss: 0.5053 - acc: 0.8133 - val_loss: 1.2162 - val_acc: 0.5867
Epoch 62/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.6202 - acc: 0.7750 - val_loss: 1.0315 - val_acc: 0.6292
Epoch 63/1000
1800/1800 [==============================] - 0s 54us/step - loss: 0.5440 - acc: 0.7939 - val_loss: 0.5295 - val_acc: 0.8042
Epoch 64/1000
1800/1800 [==============================] - 0s 54us/step - loss: 0.5398 - acc: 0.7967 - val_loss: 0.4397 - val_acc: 0.8275
Epoch 65/1000
1800/1800 [==============================] - 0s 56us/step - loss: 0.4060 - acc: 0.8567 - val_loss: 0.5557 - val_acc: 0.7942
Epoch 66/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.3970 - acc: 0.8506 - val_loss: 0.4733 - val_acc: 0.8467
Epoch 67/1000
1800/1800 [=======================

1800/1800 [==============================] - 0s 53us/step - loss: 0.0262 - acc: 0.9917 - val_loss: 0.8818 - val_acc: 0.8050
Epoch 120/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.1702 - acc: 0.9500 - val_loss: 0.0884 - val_acc: 0.9675
Epoch 121/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.0236 - acc: 0.9956 - val_loss: 0.0948 - val_acc: 0.9708
Epoch 122/1000
1800/1800 [==============================] - 0s 54us/step - loss: 0.0208 - acc: 0.9961 - val_loss: 0.1074 - val_acc: 0.9617
Epoch 123/1000
1800/1800 [==============================] - 0s 56us/step - loss: 0.0234 - acc: 0.9956 - val_loss: 0.0970 - val_acc: 0.9692
Epoch 124/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.0183 - acc: 0.9956 - val_loss: 0.3064 - val_acc: 0.9067
Epoch 125/1000
1800/1800 [==============================] - 0s 54us/step - loss: 0.0554 - acc: 0.9822 - val_loss: 0.0926 - val_acc: 0.9658
Epoch 126/1000
1800/1800 [================

1800/1800 [==============================] - 0s 55us/step - loss: 0.0034 - acc: 1.0000 - val_loss: 0.0769 - val_acc: 0.9783
Epoch 179/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.0038 - acc: 0.9994 - val_loss: 0.0800 - val_acc: 0.9750
Epoch 180/1000
1800/1800 [==============================] - 0s 61us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0748 - val_acc: 0.9750
Epoch 181/1000
1800/1800 [==============================] - 0s 81us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0873 - val_acc: 0.9725
Epoch 182/1000
1800/1800 [==============================] - 0s 87us/step - loss: 0.0038 - acc: 0.9989 - val_loss: 0.1736 - val_acc: 0.9483
Epoch 183/1000
1800/1800 [==============================] - 0s 87us/step - loss: 0.0080 - acc: 0.9978 - val_loss: 0.0805 - val_acc: 0.9775
Epoch 184/1000
1800/1800 [==============================] - 0s 80us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0786 - val_acc: 0.9750
Epoch 185/1000
1800/1800 [================

1800/1800 [==============================] - 0s 65us/step - loss: 0.0028 - acc: 0.9994 - val_loss: 0.0779 - val_acc: 0.9792
Epoch 237/1000
1800/1800 [==============================] - 0s 68us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0869 - val_acc: 0.9767
Epoch 238/1000
1800/1800 [==============================] - 0s 89us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0853 - val_acc: 0.9775
Epoch 239/1000
1800/1800 [==============================] - 0s 83us/step - loss: 7.6420e-04 - acc: 1.0000 - val_loss: 0.0820 - val_acc: 0.9783
Epoch 240/1000
1800/1800 [==============================] - 0s 57us/step - loss: 8.4720e-04 - acc: 1.0000 - val_loss: 0.0843 - val_acc: 0.9775
Epoch 241/1000
1800/1800 [==============================] - 0s 54us/step - loss: 8.5070e-04 - acc: 1.0000 - val_loss: 0.0980 - val_acc: 0.9742
Epoch 242/1000
1800/1800 [==============================] - 0s 54us/step - loss: 6.3902e-04 - acc: 1.0000 - val_loss: 0.0863 - val_acc: 0.9767
Epoch 243/1000
1800/1800 [

...

1800/1800 [==============================] - 0s 58us/step - loss: 9.2308e-06 - acc: 1.0000 - val_loss: 0.1164 - val_acc: 0.9817
Epoch 751/1000
1800/1800 [==============================] - 0s 67us/step - loss: 9.9565e-06 - acc: 1.0000 - val_loss: 0.1162 - val_acc: 0.9825
Epoch 752/1000
1800/1800 [==============================] - 0s 66us/step - loss: 2.7895e-05 - acc: 1.0000 - val_loss: 0.1192 - val_acc: 0.9817
Epoch 753/1000
1800/1800 [==============================] - 0s 52us/step - loss: 5.5423e-06 - acc: 1.0000 - val_loss: 0.1196 - val_acc: 0.9817
Epoch 754/1000
1800/1800 [==============================] - 0s 55us/step - loss: 1.2533e-05 - acc: 1.0000 - val_loss: 0.1210 - val_acc: 0.9817
Epoch 755/1000
1800/1800 [==============================] - 0s 56us/step - loss: 3.7406e-05 - acc: 1.0000 - val_loss: 0.1183 - val_acc: 0.9808
Epoch 756/1000
1800/1800 [==============================] - 0s 54us/step - loss: 7.2726e-06 - acc: 1.0000 - val_loss: 0.1188 - val_acc: 0.9808
Epoch 757/1000

1800/1800 [==============================] - 0s 70us/step - loss: 5.6931e-06 - acc: 1.0000 - val_loss: 0.1234 - val_acc: 0.9800
Epoch 808/1000
1800/1800 [==============================] - 0s 84us/step - loss: 2.2899e-05 - acc: 1.0000 - val_loss: 0.1234 - val_acc: 0.9792
Epoch 809/1000
1800/1800 [==============================] - 0s 83us/step - loss: 7.9575e-06 - acc: 1.0000 - val_loss: 0.1224 - val_acc: 0.9800
Epoch 810/1000
1800/1800 [==============================] - 0s 82us/step - loss: 5.9600e-06 - acc: 1.0000 - val_loss: 0.1227 - val_acc: 0.9792
Epoch 811/1000
1800/1800 [==============================] - 0s 84us/step - loss: 8.2866e-06 - acc: 1.0000 - val_loss: 0.1219 - val_acc: 0.9808
Epoch 812/1000
1800/1800 [==============================] - 0s 84us/step - loss: 6.0464e-06 - acc: 1.0000 - val_loss: 0.1228 - val_acc: 0.9808
Epoch 813/1000
1800/1800 [==============================] - 0s 57us/step - loss: 6.9524e-06 - acc: 1.0000 - val_loss: 0.1264 - val_acc: 0.9817
Epoch 814/1000

1800/1800 [==============================] - 0s 54us/step - loss: 3.1535e-06 - acc: 1.0000 - val_loss: 0.1388 - val_acc: 0.9808
Epoch 865/1000
1800/1800 [==============================] - 0s 53us/step - loss: 5.7344e-06 - acc: 1.0000 - val_loss: 0.1378 - val_acc: 0.9808
Epoch 866/1000
1800/1800 [==============================] - 0s 52us/step - loss: 7.0680e-06 - acc: 1.0000 - val_loss: 0.1373 - val_acc: 0.9808
Epoch 867/1000
1800/1800 [==============================] - 0s 54us/step - loss: 3.8424e-06 - acc: 1.0000 - val_loss: 0.1363 - val_acc: 0.9808
Epoch 868/1000
1800/1800 [==============================] - 0s 56us/step - loss: 0.1937 - acc: 0.9717 - val_loss: 0.9241 - val_acc: 0.8458
Epoch 869/1000
1800/1800 [==============================] - 0s 71us/step - loss: 0.2938 - acc: 0.9339 - val_loss: 0.1357 - val_acc: 0.9725
Epoch 870/1000
1800/1800 [==============================] - 0s 80us/step - loss: 0.0018 - acc: 0.9994 - val_loss: 0.1185 - val_acc: 0.9733
Epoch 871/1000
1800/1800 [

1800/1800 [==============================] - 0s 55us/step - loss: 2.7930e-05 - acc: 1.0000 - val_loss: 0.1109 - val_acc: 0.9808
Epoch 922/1000
1800/1800 [==============================] - 0s 54us/step - loss: 2.5193e-05 - acc: 1.0000 - val_loss: 0.1115 - val_acc: 0.9808
Epoch 923/1000
1800/1800 [==============================] - 0s 55us/step - loss: 4.0180e-05 - acc: 1.0000 - val_loss: 0.1187 - val_acc: 0.9800
Epoch 924/1000
1800/1800 [==============================] - 0s 54us/step - loss: 2.8774e-05 - acc: 1.0000 - val_loss: 0.1180 - val_acc: 0.9800
Epoch 925/1000
1800/1800 [==============================] - 0s 55us/step - loss: 3.1667e-05 - acc: 1.0000 - val_loss: 0.1278 - val_acc: 0.9808
Epoch 926/1000
1800/1800 [==============================] - 0s 56us/step - loss: 4.0961e-05 - acc: 1.0000 - val_loss: 0.1221 - val_acc: 0.9800
Epoch 927/1000
1800/1800 [==============================] - 0s 55us/step - loss: 1.4646e-05 - acc: 1.0000 - val_loss: 0.1220 - val_acc: 0.9792
Epoch 928/1000

1800/1800 [==============================] - 0s 69us/step - loss: 1.4288e-05 - acc: 1.0000 - val_loss: 0.1270 - val_acc: 0.9783
Epoch 979/1000
1800/1800 [==============================] - 0s 66us/step - loss: 6.5167e-06 - acc: 1.0000 - val_loss: 0.1261 - val_acc: 0.9800
Epoch 980/1000
1800/1800 [==============================] - 0s 73us/step - loss: 6.5300e-06 - acc: 1.0000 - val_loss: 0.1266 - val_acc: 0.9800
Epoch 981/1000
1800/1800 [==============================] - 0s 60us/step - loss: 5.7131e-06 - acc: 1.0000 - val_loss: 0.1263 - val_acc: 0.9800
Epoch 982/1000
1800/1800 [==============================] - 0s 55us/step - loss: 6.1190e-06 - acc: 1.0000 - val_loss: 0.1259 - val_acc: 0.9800
Epoch 983/1000
1800/1800 [==============================] - 0s 52us/step - loss: 1.7523e-05 - acc: 1.0000 - val_loss: 0.1285 - val_acc: 0.9800
Epoch 984/1000
1800/1800 [==============================] - 0s 55us/step - loss: 1.4043e-05 - acc: 1.0000 - val_loss: 0.1295 - val_acc: 0.9792
Epoch 985/1000

In [72]:
get_layer_output = K.function([model.layers[0].input], [model.layers[15].output])

In [133]:
x_train_cnn_output = get_layer_output([x_train])
x_test_cnn_output = get_layer_output([x_test])
x_val_cnn_output = get_layer_output([x_val])

In [127]:
np.array(x_train_cnn_output).shape

(1, 1800, 64)

In [135]:
x_train_cnn_output = np.array(x_train_cnn_output).reshape(len(x_train), 1, 1, 64)
x_test_cnn_output = np.array(x_test_cnn_output).reshape(len(x_test), 1, 1, 64)
x_val_cnn_output = np.array(x_val_cnn_output).reshape(len(x_val), 1, 1, 64)

In [142]:
print(x_train_cnn_output.shape)
print(x_test_cnn_output.shape)
print(x_val_cnn_output.shape)

(1800, 1, 1, 64)
(600, 1, 1, 64)
(1200, 1, 1, 64)


In [137]:
md1 = Sequential()
md1.add(Conv2D(128, (3, 3), padding='same', input_shape=(1, 1, 64)))
md1.add(Activation('relu'))
md1.add(UpSampling2D((2, 2)))

md1.add(Conv2D(128, (3, 3), padding='same'))
md1.add(Activation('relu'))
md1.add(UpSampling2D((2, 2)))

md1.add(Conv2D(64, (3, 3), padding='same'))
md1.add(Activation('relu'))
md1.add(UpSampling2D((2, 2)))

md1.add(Conv2D(64, (2, 2)))
md1.add(Activation('relu'))

md1.add(Conv2D(64, (3, 3), padding='same'))
md1.add(Activation('relu'))
md1.add(UpSampling2D((2, 2)))

md1.add(Conv2D(32, (3, 3), padding='same'))
md1.add(Activation('relu'))
md1.add(UpSampling2D((2, 2)))

md1.add(Conv2D(1, (3, 3), padding='same'))
md1.add(Activation('relu'))

In [138]:
md1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_125 (Conv2D)          (None, 1, 1, 128)         73856     
_________________________________________________________________
activation_116 (Activation)  (None, 1, 1, 128)         0         
_________________________________________________________________
up_sampling2d_76 (UpSampling (None, 2, 2, 128)         0         
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 2, 2, 128)         147584    
_________________________________________________________________
activation_117 (Activation)  (None, 2, 2, 128)         0         
_________________________________________________________________
up_sampling2d_77 (UpSampling (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 4, 4, 64)          73792     
__________

In [152]:
md1.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
md1.fit(x_train_cnn_output, x_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1, 
        validation_data=(x_val_cnn_output, x_val))


Train on 1800 samples, validate on 1200 samples
Epoch 1/1000
1800/1800 [==============================] - 1s 397us/step - loss: 0.5333 - acc: 0.0095 - val_loss: 0.5424 - val_acc: 0.0092
Epoch 2/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5341 - acc: 0.0095 - val_loss: 0.5404 - val_acc: 0.0092
Epoch 3/1000
1800/1800 [==============================] - 0s 37us/step - loss: 0.5329 - acc: 0.0096 - val_loss: 0.5430 - val_acc: 0.0092
Epoch 4/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5333 - acc: 0.0095 - val_loss: 0.5436 - val_acc: 0.0091
Epoch 5/1000
1800/1800 [==============================] - 0s 38us/step - loss: 0.5336 - acc: 0.0095 - val_loss: 0.5423 - val_acc: 0.0091
Epoch 6/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5332 - acc: 0.0096 - val_loss: 0.5487 - val_acc: 0.0090
Epoch 7/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5352 - acc: 0.0095 - val_loss: 0.5408 - val_acc:

1800/1800 [==============================] - 0s 36us/step - loss: 0.5318 - acc: 0.0096 - val_loss: 0.5451 - val_acc: 0.0091
Epoch 61/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5335 - acc: 0.0096 - val_loss: 0.5424 - val_acc: 0.0092
Epoch 62/1000
1800/1800 [==============================] - 0s 37us/step - loss: 0.5322 - acc: 0.0096 - val_loss: 0.5413 - val_acc: 0.0091
Epoch 63/1000
1800/1800 [==============================] - 0s 34us/step - loss: 0.5320 - acc: 0.0096 - val_loss: 0.5433 - val_acc: 0.0091
Epoch 64/1000
1800/1800 [==============================] - 0s 34us/step - loss: 0.5324 - acc: 0.0096 - val_loss: 0.5415 - val_acc: 0.0092
Epoch 65/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5320 - acc: 0.0096 - val_loss: 0.5427 - val_acc: 0.0092
Epoch 66/1000
1800/1800 [==============================] - 0s 37us/step - loss: 0.5321 - acc: 0.0096 - val_loss: 0.5401 - val_acc: 0.0092
Epoch 67/1000
1800/1800 [=======================

1800/1800 [==============================] - 0s 42us/step - loss: 0.5314 - acc: 0.0096 - val_loss: 0.5426 - val_acc: 0.0092
Epoch 120/1000
1800/1800 [==============================] - 0s 41us/step - loss: 0.5317 - acc: 0.0097 - val_loss: 0.5432 - val_acc: 0.0092
Epoch 121/1000
1800/1800 [==============================] - 0s 41us/step - loss: 0.5323 - acc: 0.0096 - val_loss: 0.5431 - val_acc: 0.0092
Epoch 122/1000
1800/1800 [==============================] - 0s 43us/step - loss: 0.5316 - acc: 0.0097 - val_loss: 0.5411 - val_acc: 0.0092
Epoch 123/1000
1800/1800 [==============================] - 0s 64us/step - loss: 0.5320 - acc: 0.0096 - val_loss: 0.5434 - val_acc: 0.0092
Epoch 124/1000
1800/1800 [==============================] - 0s 44us/step - loss: 0.5323 - acc: 0.0096 - val_loss: 0.5454 - val_acc: 0.0091
Epoch 125/1000
1800/1800 [==============================] - 0s 44us/step - loss: 0.5334 - acc: 0.0096 - val_loss: 0.5430 - val_acc: 0.0092
Epoch 126/1000
1800/1800 [================

1800/1800 [==============================] - 0s 37us/step - loss: 0.5309 - acc: 0.0097 - val_loss: 0.5429 - val_acc: 0.0092
Epoch 179/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5318 - acc: 0.0097 - val_loss: 0.5430 - val_acc: 0.0092
Epoch 180/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5316 - acc: 0.0097 - val_loss: 0.5444 - val_acc: 0.0092
Epoch 181/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5322 - acc: 0.0096 - val_loss: 0.5423 - val_acc: 0.0092
Epoch 182/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5313 - acc: 0.0097 - val_loss: 0.5410 - val_acc: 0.0093
Epoch 183/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5321 - acc: 0.0097 - val_loss: 0.5419 - val_acc: 0.0093
Epoch 184/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5311 - acc: 0.0097 - val_loss: 0.5448 - val_acc: 0.0093
Epoch 185/1000
1800/1800 [================

1800/1800 [==============================] - 0s 36us/step - loss: 0.5314 - acc: 0.0097 - val_loss: 0.5412 - val_acc: 0.0093
Epoch 238/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5305 - acc: 0.0097 - val_loss: 0.5413 - val_acc: 0.0093
Epoch 239/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5314 - acc: 0.0097 - val_loss: 0.5424 - val_acc: 0.0093
Epoch 240/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5311 - acc: 0.0097 - val_loss: 0.5412 - val_acc: 0.0093
Epoch 241/1000
1800/1800 [==============================] - 0s 34us/step - loss: 0.5307 - acc: 0.0097 - val_loss: 0.5423 - val_acc: 0.0093
Epoch 242/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5312 - acc: 0.0097 - val_loss: 0.5422 - val_acc: 0.0093
Epoch 243/1000
1800/1800 [==============================] - 0s 34us/step - loss: 0.5319 - acc: 0.0097 - val_loss: 0.5407 - val_acc: 0.0093
Epoch 244/1000
1800/1800 [================

1800/1800 [==============================] - 0s 37us/step - loss: 0.5300 - acc: 0.0098 - val_loss: 0.5425 - val_acc: 0.0093
Epoch 297/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5307 - acc: 0.0097 - val_loss: 0.5413 - val_acc: 0.0093
Epoch 298/1000
1800/1800 [==============================] - 0s 36us/step - loss: 0.5302 - acc: 0.0098 - val_loss: 0.5412 - val_acc: 0.0093
Epoch 299/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5299 - acc: 0.0098 - val_loss: 0.5405 - val_acc: 0.0093
Epoch 300/1000
1800/1800 [==============================] - 0s 34us/step - loss: 0.5300 - acc: 0.0097 - val_loss: 0.5447 - val_acc: 0.0092
Epoch 301/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5314 - acc: 0.0097 - val_loss: 0.5439 - val_acc: 0.0093
Epoch 302/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5311 - acc: 0.0097 - val_loss: 0.5445 - val_acc: 0.0094
Epoch 303/1000
1800/1800 [================

1800/1800 [==============================] - 0s 42us/step - loss: 0.5325 - acc: 0.0096 - val_loss: 0.5436 - val_acc: 0.0091
Epoch 356/1000
1800/1800 [==============================] - 0s 43us/step - loss: 0.5320 - acc: 0.0097 - val_loss: 0.5406 - val_acc: 0.0093
Epoch 357/1000
1800/1800 [==============================] - 0s 41us/step - loss: 0.5298 - acc: 0.0098 - val_loss: 0.5411 - val_acc: 0.0093
Epoch 358/1000
1800/1800 [==============================] - 0s 42us/step - loss: 0.5299 - acc: 0.0098 - val_loss: 0.5411 - val_acc: 0.0093
Epoch 359/1000
1800/1800 [==============================] - 0s 66us/step - loss: 0.5298 - acc: 0.0098 - val_loss: 0.5430 - val_acc: 0.0092
Epoch 360/1000
1800/1800 [==============================] - 0s 65us/step - loss: 0.5304 - acc: 0.0098 - val_loss: 0.5429 - val_acc: 0.0093
Epoch 361/1000
1800/1800 [==============================] - 0s 65us/step - loss: 0.5300 - acc: 0.0098 - val_loss: 0.5418 - val_acc: 0.0093
Epoch 362/1000
1800/1800 [================

1800/1800 [==============================] - 0s 35us/step - loss: 0.5302 - acc: 0.0098 - val_loss: 0.5434 - val_acc: 0.0094
Epoch 415/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5313 - acc: 0.0097 - val_loss: 0.5424 - val_acc: 0.0093
Epoch 416/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5296 - acc: 0.0098 - val_loss: 0.5420 - val_acc: 0.0094
Epoch 417/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5306 - acc: 0.0098 - val_loss: 0.5432 - val_acc: 0.0093
Epoch 418/1000
1800/1800 [==============================] - 0s 33us/step - loss: 0.5307 - acc: 0.0098 - val_loss: 0.5410 - val_acc: 0.0093
Epoch 419/1000
1800/1800 [==============================] - 0s 41us/step - loss: 0.5296 - acc: 0.0098 - val_loss: 0.5441 - val_acc: 0.0093
Epoch 420/1000
1800/1800 [==============================] - 0s 46us/step - loss: 0.5317 - acc: 0.0097 - val_loss: 0.5418 - val_acc: 0.0093
Epoch 421/1000
1800/1800 [================

1800/1800 [==============================] - 0s 35us/step - loss: 0.5290 - acc: 0.0098 - val_loss: 0.5411 - val_acc: 0.0093
Epoch 474/1000
1800/1800 [==============================] - 0s 35us/step - loss: 0.5293 - acc: 0.0098 - val_loss: 0.5405 - val_acc: 0.0093
Epoch 475/1000
1800/1800 [==============================] - 0s 40us/step - loss: 0.5290 - acc: 0.0098 - val_loss: 0.5414 - val_acc: 0.0092
Epoch 476/1000
1800/1800 [==============================] - 0s 52us/step - loss: 0.5293 - acc: 0.0098 - val_loss: 0.5409 - val_acc: 0.0093
Epoch 477/1000
1800/1800 [==============================] - 0s 55us/step - loss: 0.5293 - acc: 0.0098 - val_loss: 0.5425 - val_acc: 0.0092
Epoch 478/1000
1800/1800 [==============================] - 0s 54us/step - loss: 0.5296 - acc: 0.0098 - val_loss: 0.5422 - val_acc: 0.0092
Epoch 479/1000
1800/1800 [==============================] - 0s 59us/step - loss: 0.5297 - acc: 0.0098 - val_loss: 0.5401 - val_acc: 0.0093
Epoch 480/1000
1800/1800 [================

1800/1800 [==============================] - 0s 47us/step - loss: 0.5294 - acc: 0.0098 - val_loss: 0.5444 - val_acc: 0.0092
Epoch 533/1000
1800/1800 [==============================] - 0s 47us/step - loss: 0.5302 - acc: 0.0097 - val_loss: 0.5453 - val_acc: 0.0092
Epoch 534/1000
1800/1800 [==============================] - 0s 45us/step - loss: 0.5309 - acc: 0.0097 - val_loss: 0.5407 - val_acc: 0.0093
Epoch 535/1000
1800/1800 [==============================] - 0s 45us/step - loss: 0.5288 - acc: 0.0098 - val_loss: 0.5420 - val_acc: 0.0093
Epoch 536/1000
1800/1800 [==============================] - 0s 45us/step - loss: 0.5301 - acc: 0.0097 - val_loss: 0.5435 - val_acc: 0.0092
Epoch 537/1000
1800/1800 [==============================] - 0s 45us/step - loss: 0.5302 - acc: 0.0098 - val_loss: 0.5413 - val_acc: 0.0093
Epoch 538/1000
1800/1800 [==============================] - 0s 43us/step - loss: 0.5290 - acc: 0.0098 - val_loss: 0.5404 - val_acc: 0.0093
Epoch 539/1000
1800/1800 [================

In [150]:

score = md1.evaluate(x_test_cnn_output, x_test, verbose=0)
print(score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

0.5401329636573792


IndexError: invalid index to scalar variable.